In [52]:
from __future__ import division
import sys
sys.path.insert(0, "~/.local/lib/python3.6/site-packages")

import sympy
from sympy import *
from sympy.physics.quantum import *

def express(a, b, name):
    sym = symbols(name)
    sol = solve(a-sym, b)
    assert len(sol) == 1
    return (sym, sol[0])


def frange(x, y, jump):
    i = 0
    curr = x + i*jump
    
    while curr < y:
        yield curr
        i += 1
        curr = x + i*jump
    

import gmpy2
import mpmath
gmpy2.get_context().precision = 10
mpmath.mp.dps = 10

import numpy
import scipy.linalg

In [2]:
class OrthogonalKet(State, KetBase):
    
    @classmethod
    def dual_class(self):
        return OrthogonalBra
    
    def _eval_innerproduct(self, bra, **hints):

        if len(self.args) != len(bra.args):
            raise ValueError('Cannot multiply a ket that has a different number of labels.')
            
        for i in range(len(self.args)):
            diff = self.args[i] - bra.args[i]
            diff.simplify()
            
            if diff.is_nonzero:
                return Number(0)
            
            if not diff.is_zero:
                return None
            
        return Number(1)

    
class OrthogonalBra(State, BraBase):
    
    @classmethod
    def dual_class(self):
        return OrthogonalKet



from sympy.printing.pycode import AbstractPythonCodePrinter

def _print_inner_prod(self, expr):
    bra,ket = expr.args
    
    
    if not isinstance(bra, OrthogonalBra) or not isinstance(ket, OrthogonalKet):
        print(bra.func, ket.func)
        raise NotImplementedError('Only implemented for orthogonal states')
    
    if len(ket.args) != len(bra.args):
        raise ValueError('Cannot multiply a ket that has a different number of labels.')

    conditions = []

    for i in range(len(ket.args)):
        conditions.append('{a} == {b}'.format(
            a = self._print(ket.args[i]),
            b = self._print(bra.args[i])
        ))

    return '(1 if {c} else 0)'.format(
        c = ' and '.join(conditions)
    )

AbstractPythonCodePrinter._print_InnerProduct = _print_inner_prod



OKet, OBra = OrthogonalKet, OrthogonalBra

def OBraKet(a,b):
    return InnerProduct(OBra(a), OKet(b))

In [3]:
# https://stackoverflow.com/q/59523322/1137334

from sympy.core.operations import AssocOp

def apply_ccr(expr, ccr, reverse=False):
    if not isinstance(expr, Basic):
        raise TypeError("The expression to simplify is not a sympy expression.")
        
    if not isinstance(ccr, Eq):
        if isinstance(ccr, Basic):
            ccr = Eq(ccr, 0)
        else:
            raise TypeError("The canonical commutation relation is not a sympy expression.")
    
    comm = None
    
    for node in preorder_traversal(ccr):
        if isinstance(node, Commutator):
            comm = node
            break
            
    if comm is None:
        raise ValueError("The cannonical commutation relation doesn not include a commutator.")
        
    solutions = solve(ccr, comm)
    
    if len(solutions) != 1:
        raise ValueError("There are more solutions to the cannonical commutation relation.")
        
    value = solutions[0]
    
    A = comm.args[0]
    B = comm.args[1]
    
    if reverse:
        (A, B) = (B, A)
        value = -value
    
    def is_expandable_pow_of(base, expr):
        return isinstance(expr, Pow) \
            and base == expr.args[0] \
            and isinstance(expr.args[1], Number) \
            and expr.args[1] >= 1
    
    
    def walk_tree(expr):
        if isinstance(expr, Number):
            return expr
        
        if not isinstance(expr, AssocOp) and not isinstance(expr, Function):
            return expr.copy()
        
        elif not isinstance(expr, Mul):
            return expr.func(*(walk_tree(node) for node in expr.args))
        
        else:
            args = [arg for arg in expr.args]
            
            for i in range(len(args)-1):
                x = args[i]
                y = args[i+1]
                
                if B == x and A == y:
                    args = args[0:i] + [A*B - value] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if B == x and is_expandable_pow_of(A, y):
                    ypow = Pow(A, y.args[1] - 1)
                    args = args[0:i] + [A*B - value, ypow] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if is_expandable_pow_of(B, x) and A == y:
                    xpow = Pow(B, x.args[1] - 1)
                    args = args[0:i] + [xpow, A*B - value] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if is_expandable_pow_of(B, x) and is_expandable_pow_of(A, y):
                    xpow = Pow(B, x.args[1] - 1)
                    ypow = Pow(A, y.args[1] - 1)
                    args = args[0:i] + [xpow, A*B - value, ypow] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
            
            return expr.copy()
            
    
    return walk_tree(expr)
   

Basic.apply_ccr = lambda self, ccr, reverse=False: apply_ccr(self, ccr, reverse)


In [4]:
# https://stackoverflow.com/q/59524925/1137334

from sympy.core.operations import AssocOp

def apply_operator(expr, eqns):
    if not isinstance(expr, Basic):
        raise TypeError("The expression to simplify is not a sympy expression.")
    
    if not isinstance(eqns, list) and not isinstance(eqns, tuple):
        eqns = (eqns,)
    
    
    rules = []
    
    
    class Rule(object):
        operator = None
        ketSymbol = None
        result = None
        generic = False
    
    
    def is_operator(op):
        return isinstance(op, Operator) \
        or isinstance(op, Dagger) \
        and isinstance(op.args[0], Operator)
    
    
    for eqn in eqns:
        if not isinstance(eqn, Eq):
            raise TypeError("One of the equations is not a valid sympy equation.")
        
        lhs = eqn.lhs
        rhs = eqn.rhs
        
        if not isinstance(lhs, Mul) \
        or len(lhs.args) != 2 \
        or not is_operator(lhs.args[0]) \
        or not isinstance(lhs.args[1], KetBase):
            raise ValueError("The left-hand side has to be an operator applied to a ket.")
        
        rule = Rule()
        rule.operator = lhs.args[0]
        rule.ketSymbol = lhs.args[1].args[0]
        rule.result = rhs
        
        if not isinstance(rule.ketSymbol, Symbol):
            raise ValueError("The left-hand ket has to contain a simple symbol.")
        
        for ket in preorder_traversal(rhs):
            if isinstance(ket, KetBase):
                for symb in preorder_traversal(ket):
                    if symb == rule.ketSymbol:
                        rule.generic = True
                        break
                        
        rules.append(rule)
    
    
    def is_expandable_pow_of(base, expr):
        return isinstance(expr, Pow) \
            and base == expr.args[0] \
            and isinstance(expr.args[1], Number) \
            and expr.args[1] >= 1
            
            
    def is_ket_of_rule(ket, rule):
        if not isinstance(ket, KetBase):
            return False
        
        if rule.generic:
            for sym in preorder_traversal(ket):
                if sym == rule.ketSymbol:
                    return True
            return False
                
        else:
            return ket.args[0] == rule.ketSymbol
    
    
    def walk_tree(expr):
        if isinstance(expr, Number):
            return expr
        
        if not isinstance(expr, AssocOp) and not isinstance(expr, Function):
            return expr.copy()
        
        elif not isinstance(expr, Mul):
            return expr.func(*(walk_tree(node) for node in expr.args))
        
        else:
            args = [arg for arg in expr.args]
            
            for rule in rules:
                A = rule.operator
                ketSym = rule.ketSymbol
                
                for i in range(len(args)-1):
                    x = args[i]
                    y = args[i+1]

                    if A == x and is_ket_of_rule(y, rule):
                        ev = rule.result
                        
                        if rule.generic:
                            ev = ev.subs(rule.ketSymbol, y.args[0])
                        
                        args = args[0:i] + [ev] + args[i+2:]
                        return walk_tree( Mul(*args).expand() )

                    if is_expandable_pow_of(A, x) and is_ket_of_rule(y, rule):
                        xpow = Pow(A, x.args[1] - 1)
                        ev = rule.result
                        
                        if rule.generic:
                            ev = ev.subs(rule.ketSymbol, y.args[0])
                        
                        args = args[0:i] + [xpow, ev] + args[i+2:]
                        return walk_tree( Mul(*args).expand() )
                
            
            return expr.copy()
            
    
    return walk_tree(expr)
   

Basic.apply_operator = lambda self, *eqns: apply_operator(self, eqns)


In [5]:
A = Operator('ahat')
Ad = Dagger(A)
N = Ad * A

ccr = Eq( Commutator(A, Ad),  1 )

m, n = symbols('m n', integer=True, negative=False)
down = Eq( A *OKet(n), sqrt(n  )*OKet(n-1) )
up   = Eq( Ad*OKet(n), sqrt(n+1)*OKet(n+1) )

M, hbar, O = symbols('M hbar Omega', positive=True)
b = symbols('b')

x_ =     sqrt( hbar / (2*M*O) ) * (Ad + A)
p_ = I * sqrt( hbar * M*O / 2 ) * (Ad - A)

assert I * hbar == Commutator(x_, p_).doit().expand().apply_ccr(ccr)

In [6]:
a, b = symbols('a b')
p, x = Operator('phat'), Operator('xhat')
H = p**2 / (2*M)  -  (a*x**2)/2  +  (b*x**4)/2
H

-a*xhat**2/2 + b*xhat**4/2 + phat**2/(2*M)

In [7]:
Hmn = OBra(m) * H * OKet(n)
Hmn = Hmn.subs(x, x_).subs(p, p_)
Hmn = Hmn.expand()
Hmn = Hmn.apply_operator(up, down)
Hmn = Hmn.subs(M,1).subs(O,1).subs(b,1)
Hmn = qapply(Hmn)
Hmn = eval(srepr(Hmn))
Hmn = Hmn.collect(OBraKet(m,n-4))
Hmn = Hmn.collect(OBraKet(m,n-2))
Hmn = Hmn.collect(OBraKet(m,n))
Hmn = Hmn.collect(OBraKet(m,n+2))
Hmn = Hmn.collect(OBraKet(m,n+4))
Hmn

hbar**2*sqrt(n)*sqrt(n - 3)*sqrt(n - 2)*sqrt(n - 1)*<m|n - 4>/8 + hbar**2*sqrt(n + 1)*sqrt(n + 2)*sqrt(n + 3)*sqrt(n + 4)*<m|n + 4>/8 + (-a*hbar*sqrt(n)*sqrt(n - 1)/4 + hbar**2*n**(3/2)*sqrt(n - 1)/2 - hbar**2*sqrt(n)*sqrt(n - 1)/4 - hbar*sqrt(n)*sqrt(n - 1)/4)*<m|n - 2> + (-a*hbar*sqrt(n + 1)*sqrt(n + 2)/4 + hbar**2*n*sqrt(n + 1)*sqrt(n + 2)/2 + 3*hbar**2*sqrt(n + 1)*sqrt(n + 2)/4 - hbar*sqrt(n + 1)*sqrt(n + 2)/4)*<m|n + 2> + (-a*hbar*n/2 - a*hbar/4 + 3*hbar**2*n**2/4 + 3*hbar**2*n/4 + 3*hbar**2/8 + hbar*n/2 + hbar/4)*<m|n>

In [8]:
Hmn_refined = \
    hbar**2 / 8  *  sqrt(n*(n - 1)*(n - 2)*(n - 3)) * OBraKet(m,n-4) + \
    hbar**2 / 8  *  sqrt((n + 1)*(n + 2)*(n + 3)*(n + 4)) * OBraKet(m,n+4) + \
    hbar/4 * sqrt(n*(n - 1)) * (2 *hbar *n - hbar - a - 1) * OBraKet(m,n-2) + \
    hbar/4 * sqrt((n+1)*(n+2)) * (-a + 2*hbar*n + 3*hbar - 1) * OBraKet(m,n+2) + \
    hbar/8 * (- 4*a*n - 2*a + 6*hbar *n**2 + 6*hbar*n + 3*hbar + 4*n + 2) * OBraKet(m,n)
    
assert 0 == (Hmn - Hmn_refined).subs(n,n+4).simplify()

Hmn = Hmn_refined
Hmn

hbar**2*sqrt(n)*sqrt((n - 3)*(n - 2)*(n - 1))*<m|n - 4>/8 + hbar**2*sqrt(n + 1)*sqrt(n + 2)*sqrt(n + 3)*sqrt(n + 4)*<m|n + 4>/8 + hbar*sqrt(n)*sqrt(n - 1)*(-a + 2*hbar*n - hbar - 1)*<m|n - 2>/4 + hbar*sqrt(n + 1)*sqrt(n + 2)*(-a + 2*hbar*n + 3*hbar - 1)*<m|n + 2>/4 + hbar*(-4*a*n - 2*a + 6*hbar*n**2 + 6*hbar*n + 3*hbar + 4*n + 2)*<m|n>/8

In [59]:
Hmn_fun = lambdify( (m,n,a,hbar), re(Hmn), modules='sympy' )
Hmn_prec = lambda i,j,a,h: sympy.N(Hmn_fun(i,j,a,h), 10)

def H_mat(N,a,h):
    
    mat = mpmath.matrix(N)
    
    for i in range(N):
        for j in range(N):
            mat[i,j] = mpmath.mpf(Hmn_prec(i,j,a,h))
            
    return mat
    
    
def H_eigvals(N,a,h):
    ev,_ = mpmath.eig(H_mat(N,a,h))
    ev.sort()
    return ev


def H_mat_fast(N,a,h):
    
    mat = numpy.ndarray( (N,N) )
    
    for i in range(N):
        for j in range(N):
            mat[i,j] = float(Hmn_prec(i,j,a,h))
            
    return mat


def H_eigvals_fast(N,a,h):
    ev = [ n.real for n in scipy.linalg.eigvals(H_mat_fast(N,a,h)) ]
    ev.sort()
    return ev

In [61]:
def energies(k,a,h,prec,Nmin=None):
    maxdiff = 10**(-prec-3)
    ev_now = []
    ev_nxt = H_eigvals(k,a,h)
    
    if Nmin is None or Nmin <= k:
        Nmin = 2*k
    
    for N in range(Nmin,100*k,k):
        ev_now = ev_nxt
        ev_nxt = H_eigvals(N,a,h)
        
        if all([ abs(ev_nxt[i] - ev_now[i]) < maxdiff for i in range(k) ]):
            print("N = "+str(N))
            break
        
    return [ev_nxt[i] for i in range(k)]

def energiesf(k,a,h,prec,Nmin=None):
    return [ mpmath.nstr(ev, prec) for ev in energies(k,a,h,prec,Nmin) ]

def energies_fast(k,a,h,prec,Nmin=None):
    maxdiff = 10**(-prec-3)
    ev_now = []
    ev_nxt = H_eigvals_fast(k,a,h)
    
    if Nmin is None or Nmin <= k:
        Nmin = 2*k
    
    for N in range(Nmin,100*k,k):
        ev_now = ev_nxt
        ev_nxt = H_eigvals_fast(N,a,h)
        
        if all([ abs(ev_nxt[i] - ev_now[i]) < maxdiff for i in range(k) ]):
            print("N = "+str(N))
            break
        
    return [ev_nxt[i] for i in range(k)]

def energies_faster(k,a,h,prec,N):
    ev = H_eigvals_fast(N,a,h)
    return [ev[i] for i in range(k)]

In [32]:
f = open("3c_eigvals.csv", "w")

ev = energiesf(4,1,0.03,5,24)
f.write("ea, eb, ec, ed \n")
f.write(", ".join(ev))

f.close()

ev

N = 32


['-0.10426', '-0.10426', '-0.064909', '-0.0649']

In [104]:
f = open("3c_hbar_prec45.dat", "w", buffering=1)

for h in frange(0.0, 0.03, 0.0003):
    print(h)
    arr = [h] + energies_faster(8,1,h,3,45)
    f.write(" ".join( [str(n) for n in arr] ) + "\n")
    
f.close()

0.0
0.0003
0.0006
0.0009
0.0012
0.0014999999999999998
0.0018
0.0021
0.0024
0.0026999999999999997
0.0029999999999999996
0.0032999999999999995
0.0036
0.0039
0.0042
0.0045
0.0048
0.0050999999999999995
0.005399999999999999
0.005699999999999999
0.005999999999999999
0.006299999999999999
0.006599999999999999
0.006899999999999999
0.0072
0.0075
0.0078
0.0081
0.0084
0.0087
0.009
0.0093
0.0096
0.009899999999999999
0.010199999999999999
0.010499999999999999
0.010799999999999999
0.011099999999999999
0.011399999999999999
0.011699999999999999
0.011999999999999999
0.012299999999999998
0.012599999999999998
0.012899999999999998
0.013199999999999998
0.013499999999999998
0.013799999999999998
0.014099999999999998
0.0144
0.0147
0.015
0.0153
0.0156
0.015899999999999997
0.0162
0.016499999999999997
0.0168
0.017099999999999997
0.0174
0.017699999999999997
0.018
0.018299999999999997
0.0186
0.018899999999999997
0.0192
0.0195
0.019799999999999998
0.0201
0.020399999999999998
0.0207
0.020999999999999998
0.0213
0.02159

In [73]:
energies_fast(8,1,0.03,3,40)

N = 48


[-0.104260978339396,
 -0.10426091324244724,
 -0.06490926200816749,
 -0.0649001679985942,
 -0.029792399225931705,
 -0.029346956539089777,
 -0.0034621291903542705,
 0.0027228655845642612]

In [74]:
energies_fast(8,1,1,3,40)

N = 64


[0.3288265025909685,
 1.4172681010574522,
 3.081950628485787,
 5.019323060371725,
 7.186203252293968,
 9.542857342858646,
 12.064037747207951,
 14.731427963167326]

In [75]:
energies_fast(8,1,0.01,3,40)

N = 64


[-0.11797975697376543,
 -0.11797975697357879,
 -0.10414903196996549,
 -0.10414903196996408,
 -0.09064955583143097,
 -0.09064955583122299,
 -0.07750710806376218,
 -0.0775071080622314]

In [90]:
expr = ( Bra('m') + 2*Bra('n') ) * ( Ket('n') - 3*Ket('m') )
expr = expr.subs( Bra('m'), 2*Bra('n') ).subs( Ket('m'), 2*Ket('n') )
expr = expr.expand()
expr = qapply(expr)
expr

-20*<n|n>